Copyright **`(c)`** 2021 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see 'LICENCE.md' for details.

# Connect 4

In [ ]:
from collections import Counter
import numpy as np
import math

In [ ]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"

## Basic Functions

In [ ]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )

## MinMax

In [ ]:
def other(player):
    return player * -1;

def score_is_better(score, current_best, player):
    if player == 1:
        return score > current_best
    else:
        return score < current_best

def update_alpha_beta(alpha, beta, best_score, player):
    if player == 1:
        return max(alpha, best_score), beta
    else:
        return alpha, min(beta, best_score)

def count_streaks(board, player, length):
    """Checks how many length-piece lines the player has"""
    return (
        sum(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + length)) for n in range(COLUMN_HEIGHT - length + 1))
        )
        + sum(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + length)) for n in range(NUM_COLUMNS - length + 1))
        )
        + sum(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + length), range(co, co + length))
                for ro in range(0, NUM_COLUMNS - length + 1)
                for co in range(0, COLUMN_HEIGHT - length + 1)
            )
        )
        + sum(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + length), range(co + length - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - length + 1)
                for co in range(0, COLUMN_HEIGHT - length + 1)
            )
        )
    )

def eval(board):
    score = 0
    score += 20 * count_streaks(board, 1, 3)
    score -= 20 * count_streaks(board, -1, 3)
    score += 5 * count_streaks(board, 1, 2)
    score -= 5 * count_streaks(board, -1, 2)
    print(score)
    return score

def check_win(score, player):
    return (player * score) == math.inf

def minmax(board, depth, alpha, beta, player):
    if four_in_a_row(board, player):
        return None, (player * math.inf)
    elif four_in_a_row(board, other(player)):
        return None, (other(player) * math.inf)
    if depth == 0:
        return None, eval(board)
    best_score = other(player) * math.inf
    best_move = None
    for c in valid_moves(board):
        play(board, c, player)
        score = minmax(board, depth - 1, alpha, beta, other(player))[1]
        take_back(board, c)
        if score_is_better(score, best_score, player):
            best_score = score
            best_move = c
        alpha, beta = update_alpha_beta(alpha, beta, best_score, player)
        if alpha >= beta:
            break
    if best_move is None: # tie
        best_score = 0
    return best_move, best_score


## Montecarlo Evaluation

In [ ]:
def _mc(board, player):
    p = -player
    while valid_moves(board):
        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)
        if four_in_a_row(board, p):
            return p
    return 0


def montecarlo(board, player):
    montecarlo_samples = 100
    cnt = Counter(_mc(np.copy(board), player) for _ in range(montecarlo_samples))
    return (cnt[1] - cnt[-1]) / montecarlo_samples


def eval_board(board, player):
    if four_in_a_row(board, 1):
        # Alice won
        return 1
    elif four_in_a_row(board, -1):
        # Bob won
        return -1
    else:
        # Not terminal, let's simulate...
        return montecarlo(board, player)

## Example

In [ ]:
board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
play(board, 3, 1)
play(board, 0, -1)
play(board, 4, 1)
play(board, 0, -1)
play(board, 5, 1)
print(board)
eval_board(board, 1)

In [ ]:
board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
player = 1
alpha = -math.inf
beta = math.inf

while True:
    next_move, score = minmax(board, 5, alpha, beta, player)
    if check_win(score, player):
        break
    play(board, next_move, player)
    print(board)
    print()
    player = other(player)